### 2024/01/18 重複する座標を除いたうえでコサイン類似度を求め、次元圧縮する

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox 
from sklearn import datasets
import seaborn as sns
import umap
import sklearn
from sklearn.manifold import TSNE
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from view_function import ViewFunction
vf = ViewFunction()
from preprocessed_data_plusinfo import PreprocessedDataPlusInfo


palette = ['red', 'blue', 'green', 'yellow', 'black']

In [2]:
box_abct_nopt = pd.read_pickle('../temp/20240117_box_abct_nopt_rmoverlapcoord.pkl')

##### ▶紙箱の直線のみを入力
（データ数が大きすぎるとUMAPのエラーが出るので、データ数を941*4 = 3768にする）

In [4]:
size = 942 * 4
seed = 1
input_data = box_abct_nopt.query('shape_int == 1').sample(n=size, random_state=1)
input_nparray = np.array([np.array(nparr) for nparr in input_data['cos_sim'].to_numpy()])
input_nparray.shape

(3768, 941)

In [15]:
for n_neighbors in [4000, 5000]:
    mapper = umap.UMAP(n_components=2, 
                        n_neighbors=n_neighbors,
                        min_dist=0.1,
                        metric = "euclidean"
                        )
    umap_result = mapper.fit_transform(input_nparray)

    input_data['axis0'] = umap_result[:, 0:1]
    input_data['axis1'] = umap_result[:, 1:2]
    
    title = f'strt_3768data_cossim_nneighbors_{n_neighbors}'
    plt.rcParams['font.family'] = 'MS Gothic'
    FIG_SIZE = (10, 10)
    fig, ax = plt.subplots(figsize=(FIG_SIZE[0],FIG_SIZE[1]))
    ax.set_xlabel('axis0')
    ax.set_ylabel('axis1')
    ax.set_title(title)
    ax.grid(True, alpha=0.5)
    
    for ax0, ax1, shape_int in zip(input_data['axis0'], input_data['axis1'], input_data['shape_int']):
        if shape_int == 1:
            plt.scatter(ax0, ax1, color='blue', alpha=0.2)
        elif shape_int == 2:
            plt.scatter(ax0, ax1, color='green', alpha=0.2)
        elif shape_int == 3:
            plt.scatter(ax0, ax1, color='yellow', alpha=0.2)
        else:
            plt.scatter(ax0, ax1, color='black', alpha=0.2)
            
    plt.scatter(input_data['axis0'], input_data['axis1'], color='blue', alpha=0.5)
    plt.savefig(f'{title}_scatter.png')
    plt.close()
    input_data.to_csv(f'../temp/{title}.tsv', sep='\t', encoding='shift-jis')
    
    print(f'finish | {n_neighbors}')

c:\Users\ispec\MyWorkSpace\python\drawing_2023\.venv\lib\site-packages\umap\umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


finish | 4000


c:\Users\ispec\MyWorkSpace\python\drawing_2023\.venv\lib\site-packages\umap\umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


finish | 5000


##### ▶すべての線種を入力してみる
各線種を942個に統一

In [5]:
box_abct_nopt_st = box_abct_nopt[box_abct_nopt['shape_int'] == 1]
box_abct_nopt_cu = box_abct_nopt[box_abct_nopt['shape_int'] == 2]
box_abct_nopt_co = box_abct_nopt[box_abct_nopt['shape_int'] == 3]
box_abct_nopt_ha = box_abct_nopt[box_abct_nopt['shape_int'] == 4]

# 線種ごとのデータ数の最小値 size(hatching)で統一
size = 942
seed = 1

box_abct_nopt_st_sample = box_abct_nopt_st.sample(n=size, random_state=seed)
box_abct_nopt_cu_sample = box_abct_nopt_cu.sample(n=size, random_state=seed)
box_abct_nopt_co_sample = box_abct_nopt_co.sample(n=size, random_state=seed)
box_abct_nopt_ha_sample = box_abct_nopt_ha.sample(n=size, random_state=seed)

all_shape_random = pd.concat([box_abct_nopt_st_sample, box_abct_nopt_cu_sample, box_abct_nopt_co_sample, box_abct_nopt_ha_sample], axis=0)
all_shape_random_shuffle = sklearn.utils.shuffle(all_shape_random, random_state=seed)

input_data = all_shape_random_shuffle.copy()
input_nparray = np.array([np.array(nparr) for nparr in input_data['cos_sim'].to_numpy()])
input_data


,stroke_id,time,shape_str,pressure_avg,shape_int,len_mm,ptx_mm,pty_mm,remove_index,ptx_mm_slide_expand,...,140*140_img,140*140_img_vector,drawing_id,year,month,day,motif,times,name,cos_sim
2700,2700,1277,Straight,102.666667,1,114.888922,"[125.47500000000002, 127.53750000000002, 130.3...","[202.2375, 202.01250000000002, 202.20000000000...",[],"[0.0, 0.4617137374827735, 0.923427474965547, 1...",...,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",292,2015,4,16,紙箱,1,冨山恭子,"[1.0000000000000002, 1.0, 1.0, 1.0000000000000..."
1476,1476,1134,Complex,98.814815,3,103.597450,"[72.67500000000001, 74.77500000000003, 75.5250...","[203.77500000000003, 205.95000000000002, 206.3...",[],"[190.7339449541283, 192.06591913013926, 193.39...",...,"[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",50,2013,5,13,紙箱,2,高田麻央,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99999999..."
476,476,661,Hatching,87.700000,4,117.137892,"[340.8, 339.4875, 337.575, 334.98749999999995,...","[58.499999999999986, 58.72500000000001, 59.662...",[],"[420.0, 418.2110259218692, 416.4220518437384, ...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3,2013,4,18,紙箱,1,井原亜利紗,"[1.0000000000000002, 1.0000000000000002, 1.0, ..."
122,122,334,Complex,58.695652,3,86.603068,"[149.025, 147.86249999999998, 146.549999999999...","[113.2875, 112.725, 111.975, 111.1875, 111.149...",[],"[149.89399293286291, 146.60777385159028, 143.3...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, ...",21,2013,4,18,紙箱,1,高田麻央,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
47,47,57,Simple curve,44.000000,2,16.668141,"[58.91250000000002, 58.53750000000002, 58.3500...","[177.90000000000003, 179.73750000000004, 181.3...",[],"[21.954545454545745, 21.864493996569728, 21.77...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...",50,2013,5,13,紙箱,2,高田麻央,"[1.0, 1.0, 1.0, 0.9999999999999999, 1.0, 1.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,1198,492,Complex,113.217391,3,151.889159,"[214.95000000000002, 216.5625, 215.6625, 206.4...","[178.8, 178.875, 178.875, 178.8, 178.95, 180.0...",[],"[365.720338983051, 367.6334745762713, 369.5466...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",308,2015,4,27,紙箱,2,冨山恭子,"[1.0000000000000002, 1.0, 1.0, 1.0, 1.0, 1.000..."
371,371,154,Straight,31.076923,1,49.852583,"[298.3875, 311.925, 324.0, 327.1875, 330.075, ...","[155.54999999999998, 148.8375, 142.275, 140.66...",[],"[0.0, 0.45778985507272213, 0.9155797101448907,...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",955,2016,5,2,紙箱,1,saito,"[0.9999999999999998, 1.0, 1.0, 1.0, 1.0, 1.000..."
1403,1403,811,Simple curve,90.800000,2,38.976669,"[295.2, 297.45, 297.375, 290.28749999999997, 2...","[141.0375, 141.225, 141.375, 141.8624999999999...",[],"[392.4590163934426, 392.9597615499257, 393.460...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",957,2022,6,13,紙箱,3,saito,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
298,298,152,Straight,91.714286,1,57.150758,"[140.925, 144.03750000000002, 142.5, 126.52500...","[155.70000000000002, 153.9, 154.9125, 168.375,...",[],"[386.64114832535876, 387.2065525910306, 387.77...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",308,2015,4,27,紙箱,2,冨山恭子,"[1.0, 1.0, 1.0000000000000002, 1.0, 1.0, 1.000..."


In [6]:
for n_neighbors in [1000, 2000, 3000, 4000, 5000]:
    mapper = umap.UMAP(n_components=2, 
                        n_neighbors=n_neighbors,
                        min_dist=0.1,
                        metric = "euclidean"
                        )
    umap_result = mapper.fit_transform(input_nparray)

    input_data['axis0'] = umap_result[:, 0:1]
    input_data['axis1'] = umap_result[:, 1:2]
    
    title = f'allshape_3768data_cossim_nneighbors_{n_neighbors}'
    plt.rcParams['font.family'] = 'MS Gothic'
    FIG_SIZE = (10, 10)
    fig, ax = plt.subplots(figsize=(FIG_SIZE[0],FIG_SIZE[1]))
    ax.set_xlabel('axis0')
    ax.set_ylabel('axis1')
    ax.set_title(title)
    ax.grid(True, alpha=0.5)
    
    for ax0, ax1, shape_int in zip(input_data['axis0'], input_data['axis1'], input_data['shape_int']):
        if shape_int == 1:
            plt.scatter(ax0, ax1, color='blue', alpha=0.2)
        elif shape_int == 2:
            plt.scatter(ax0, ax1, color='green', alpha=0.2)
        elif shape_int == 3:
            plt.scatter(ax0, ax1, color='yellow', alpha=0.2)
        else:
            plt.scatter(ax0, ax1, color='black', alpha=0.2)
            
    # plt.scatter(input_data['axis0'], input_data['axis1'], color='blue', alpha=0.5)
    plt.savefig(f'{title}_scatter.png')
    plt.close()
    input_data.to_csv(f'../temp/{title}.tsv', sep='\t', encoding='shift-jis')
    
    print(f'finish | {n_neighbors}')

finish | 1000
finish | 2000
finish | 3000


c:\Users\ispec\MyWorkSpace\python\drawing_2023\.venv\lib\site-packages\umap\umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


finish | 4000


c:\Users\ispec\MyWorkSpace\python\drawing_2023\.venv\lib\site-packages\umap\umap_.py:2433: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


finish | 5000
